In [1]:
import datetime
import json
from google.cloud import firestore
from google.oauth2 import service_account

In [2]:
def authenticate_to_firestore():
    """Authenticates to Firestore and returns a client."""
    # read the '../firestore-key.json' file
    key_dict = json.load(open('../firestore-key.json'))
    creds = service_account.Credentials.from_service_account_info(key_dict)
    db = firestore.Client(credentials=creds, project="dsm2023isw")
    return db

db = authenticate_to_firestore()

In [3]:
docs_all = db.collection("session_states").stream()

In [9]:
# docs = (
#     db.collection("session_states")
#     #.where(filter=FieldFilter("capital", "==", True))
#     .stream()
# )

In [4]:
docs_list = []
for doc in docs_all:
    # print(f"{doc.id} => {doc.to_dict()}")
    # my_dict = { el.id: el.to_dict() for el in doc }
    docs_list.append(doc.to_dict())

print(len(docs_list))

# Save the data to a json file
with open("session_states.json", "w") as f:
    json.dump(docs_list, f)

3635


In [5]:
# find all the unique session_id in the session_states.json file
unique_session_ids = []
for i in range(len(docs_list)):
    #print(docs_list[i]['session_id'])
    if docs_list[i]['session_id'] not in unique_session_ids:
        unique_session_ids.append(docs_list[i]['session_id'])

print(len(unique_session_ids))


# count the number of elements in the json file that share the same session_id



471


Need to filter out:
Has timestamp earlier than 2023-10-04 07:00
Has timestamp later than 2023-10-04 14:15/15:00
Has group == "Select" and only one entry


In [6]:
# Filter the data to only include the data during the workshop
during_ws = []
for i in range(len(docs_list)):
    # str date as datetime date
    timestamp = datetime.datetime.strptime(docs_list[i]['timestamp'], '%Y-%m-%d %H:%M:%S')
    ws_start = datetime.datetime(2023, 10, 4, 7, 0, 0)
    ws_end = datetime.datetime(2023, 10, 4, 15, 0, 0)
    #print(docs_list[i]['session_id'])
    if (timestamp > ws_start and timestamp < ws_end):
        during_ws.append(docs_list[i])
print(len(during_ws))

2870


In [7]:
# Filter the data to only include the data after the workshop
after_ws = []
for i in range(len(docs_list)):
    # str date as datetime date
    timestamp = datetime.datetime.strptime(docs_list[i]['timestamp'], '%Y-%m-%d %H:%M:%S')
    ws_start = datetime.datetime(2023, 10, 4, 7, 0, 0)
    ws_end = datetime.datetime(2023, 10, 4, 15, 0, 0)
    #print(docs_list[i]['session_id'])
    if (timestamp > ws_end):
        after_ws.append(docs_list[i])
print(len(after_ws))

219


In [16]:
# count the number of elements in the json file that share the same session_id

for i in range(len(unique_session_ids)):
    count = 0
    for j in range(len(docs_list)):
        if docs_list[j]['session_id'] == unique_session_ids[i]:
            count += 1
    if count == 0:
        print(f"Session_id: {unique_session_ids[i]} has {count} elements")

In [17]:
group_select = []
for i in range(len(docs_list)):
    if docs_list[i]['group'] == 'Select':
        group_select.append(docs_list[i])

print(len(group_select))

328
